In [27]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic

In [28]:
import gradio as gr # oh yeah!

In [29]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [30]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()
claude = anthropic.Anthropic()

In [31]:
# A generic system message - no more snarky adversarial AIs!
system_message = "You are a helpful assistant"

In [32]:

def stream_gpt(prompt, model_version):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model=model_version,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [33]:
def stream_claude(prompt, model_version):
    result = claude.messages.stream(
        model=model_version,
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [34]:
# function using both dropdown values
def stream_model(message, model_family, model_version):
    if model_family == 'GPT':
        result =  stream_gpt(message, model_version)
    elif model_family == 'Claude':
        result = stream_claude ( message, model_version)
    yield from result

In [35]:

# Define available model versions
model_versions = {
    "GPT": ["gpt-4o-mini", "gpt-4.1-mini", "gpt-4.1-nano", "gpt-4.1", "o3-mini"],
    "Claude": ["claude-3-haiku-20240307", "claude-3-opus-20240229", "claude-3-sonnet-20240229"]
}

# Update second dropdown options based on first dropdown selection
def update_model_versions(selected_model_family):
    return gr.update(choices=model_versions[selected_model_family], value=model_versions[selected_model_family][0])


with gr.Blocks() as demo:
    model_family_dropdown = gr.Dropdown(
            label="Select Model Family",
            choices=["GPT", "Claude"],
            value="GPT"
        )
    model_version_dropdown = gr.Dropdown(
            label="Select Model Version",
            choices=model_versions["GPT"],  # Default choices
            value=model_versions["GPT"][0]
        )
    
    message_input = gr.Textbox(label="Your Message")
    output = gr.Markdown(label="Response")

    # Bind logic to update model version dropdown
    model_family_dropdown.change(
        fn=update_model_versions,
        inputs=model_family_dropdown,
        outputs=model_version_dropdown
    )

    # Launch function on submit
    submit_btn = gr.Button("Submit")
    submit_btn.click(
        fn=stream_model,
        inputs=[message_input, model_family_dropdown, model_version_dropdown],
        outputs=output
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.


/var/folders/sh/yytd3s6n3wd6952jnw97_v940000gn/T/ipykernel_7803/4165844704.py:7: DeprecationWarning: The model 'claude-3-opus-20240229' is deprecated and will reach end-of-life on January 5th, 2026.
Please migrate to a newer model. Visit https://docs.anthropic.com/en/docs/resources/model-deprecations for more information.
  yield from result
Traceback (most recent call last):
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/